<a href="https://colab.research.google.com/github/Antsistra/Rute-Rumah-Sakit-Terdekat-A-Stars/blob/main/Rute_Rumah_sakit_Terdekat_dari_Perumahan_X_Menggunakan_A_(A_Stars)_Method_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Library and Packages

In [45]:
import folium as fl
import pandas as pd
import numpy as np

Generate Dataset

In [46]:
data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    "Titik": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"],
    "Lokasi": [
        "Citraland", "Jl Raya Sepang", "Jl Ciracas", "Jl Kolonel Tubagus Suwandi",
        "Jl Veteran", "Jl Kh Abdul Hadi", "Jl Kejaksaan I", "Jl Kh Sochan",
        "Jl Kh abdul fatah hasan", "Jl Jend Sudirman", "Rs Sari Asih Serang"
    ],
    "Koordinat": [
        "-6.131067, 106.132513", "-6.133913, 106.134790", "-6.127008, 106.144934",
        "-6.127864, 106.146556", "-6.116890, 106.153907", "-6.125760, 106.159867",
        "-6.123619, 106.156434", "-6.122556, 106.164502", "-6.124513, 106.171925",
        "-6.120478, 106.173673", "-6.118677, 106.181270"
    ]
}

data['X'] = [float(coord.split(", ")[0]) for coord in data['Koordinat']]
data['Y'] = [float(coord.split(", ")[1]) for coord in data['Koordinat']]

df = pd.DataFrame(data)

df.to_csv("dataset/koordinat.csv", index=False)


Mengecek Hasil Dataset

In [ ]:
dataset = pd.read_csv('dataset/koordinat.csv')
dataset

Showing Maps and Legends

In [ ]:

_map = fl.Map(location=[-6.1236296, 106.1556118], zoom_start=14)


for index, row in dataset.iterrows():
    fl.Marker(
        location=[row['X'], row['Y']],
        popup=row['Lokasi'],
        tooltip=row['Titik'],
    ).add_to(_map)
    fl.Marker(location=[-6.131067,106.132513], icon=fl.Icon(color='green',icon='home',prefix='fa')).add_to(_map)
    fl.Marker(location=[-6.118677,106.181270], icon=fl.Icon(color='red',icon='plus-square',prefix='fa')).add_to(_map)

_map


Implementasi Perhitungan Ecludean untuk menentukan H(n)


In [48]:


file_path = 'dataset/koordinat.csv'
data = pd.read_csv(file_path)

pairs = ['AB', 'BC', 'BD', 'CE', 'DF', 'EG', 'EJ', 'FG', 'FH', 'FI', 'HJ', 'IJ', 'JK']

coordinates = {row['Titik']: (row['X'], row['Y']) for _, row in data.iterrows()}

def euclidean_distance(point1, point2):
    x1, y1 = coordinates[point1]
    x2, y2 = coordinates[point2]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

distances = []
for pair in pairs:
    point1, point2 = pair[0], pair[1]
    distance = euclidean_distance(point1, point2)
    distances.append((pair, distance))

results_df = pd.DataFrame(distances, columns=['Titik', 'nilai_heuristic'])

results_df.to_csv('dataset/heuristic.csv', index=False)


Nilai H(n)

In [ ]:
dataset = pd.read_csv('dataset/nilai_heuristic.csv')
dataset